In [1]:
import tensorflow as tf
import numpy as np
import datetime, time

In [24]:
def genObs(length, dict_size=3):
    slen = length // 2
    base = np.random.choice(range(dict_size), size=slen)
    if slen * 2 < length:
        return np.concatenate([base, np.random.choice(range(dict_size), size=1), np.array(list(reversed(base)))])
    return np.concatenate([base, np.array(list(reversed(base)))])

def genSample(num, length=20):
    x = [genObs(length=length) for _ in range(num)]
    return np.array(x)

def randomBatch(tensorTuple, batchSize=64):
    ids = np.random.choice(range(tensorTuple[0].shape[0]), batchSize)
    return (x[ids,] for x in tensorTuple)

def shuffleBatches(tensorTuple, batchSize=64):
    if type(tensorTuple) is list or type(tensorTuple) is tuple: 
        ids = list(range(tensorTuple[0].shape[0]))
        np.random.shuffle(ids)
        for i in range(0,len(ids),batchSize):
            lst = min(len(ids), i + batchSize)
            yield (np.array(x[ids[i:lst],]) for x in tensorTuple)
    else:
        ids = list(range(tensorTuple.shape[0]))
        np.random.shuffle(ids)
        for i in range(0,len(ids),batchSize):
            lst = min(len(ids), i + batchSize)
            yield np.array(tensorTuple[ids[i:lst],])

In [83]:
train_x = genSample(100000, length=20)
valid_x = genSample(10000, length=20)
valid_x[:10]

array([[0, 0, 2, 0, 1, 1, 2, 1, 2, 0, 0, 2, 1, 2, 1, 1, 0, 2, 0, 0],
       [1, 0, 0, 2, 1, 2, 0, 2, 1, 0, 0, 1, 2, 0, 2, 1, 2, 0, 0, 1],
       [1, 0, 1, 0, 0, 2, 1, 2, 0, 0, 0, 0, 2, 1, 2, 0, 0, 1, 0, 1],
       [2, 2, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 2, 2],
       [2, 1, 0, 2, 0, 0, 1, 2, 1, 2, 2, 1, 2, 1, 0, 0, 2, 0, 1, 2],
       [2, 1, 0, 2, 0, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 0, 2, 0, 1, 2],
       [0, 0, 0, 2, 1, 1, 1, 1, 0, 2, 2, 0, 1, 1, 1, 1, 2, 0, 0, 0],
       [2, 2, 0, 0, 1, 1, 1, 2, 0, 1, 1, 0, 2, 1, 1, 1, 0, 0, 2, 2],
       [2, 1, 2, 1, 0, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 0, 1, 2, 1, 2],
       [0, 1, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 2, 0, 1, 0]])

In [88]:
SEQ_LEN = 20
LOSS_CUT = 9 #start loss evaluation with 9th symbol
DICT_SIZE = 3
RNN_SIZE = [20]

tf.reset_default_graph()

InnerCell = lambda n: tf.nn.rnn_cell.GRUCell(num_units=n, activation=tf.nn.elu)
rnnCell = tf.nn.rnn_cell.MultiRNNCell([InnerCell(s) for s in RNN_SIZE], state_is_tuple=True)
#rnnCell = tf.nn.rnn_cell.BasicRNNCell(10, activation=tf.nn.elu)


tfi_x = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.int64)

tfX = tf.one_hot(tfi_x, DICT_SIZE, dtype=tf.float32)
tfY = tfX[:,1:,:]

tfHS, _ = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32, time_major=False)
tfH = tfHS#[-1] #get latest layer in RNN

tfHOut0 = tf.layers.dense(tfH, DICT_SIZE)
tfHOut = tfHOut0[:,:-1,:] #remove last forecast

tfHLoss0 = tf.nn.softmax_cross_entropy_with_logits(labels=tfY, logits=tfHOut) #should be like batch_size x SEQ_LEN-1
tfLoss = tf.reduce_mean(tfHLoss0[:, LOSS_CUT:])
tfTrain = tf.train.AdamOptimizer(1e-3).minimize(tfLoss)

tfPredicted = tf.cast(tf.argmax(tfHOut, axis=2)[:,LOSS_CUT:], dtype=tf.int64)
tfActual = tfi_x[:, (LOSS_CUT+1):]

tfAccuracyChar = tf.reduce_mean(tf.cast(tf.equal(tfActual, tfPredicted), dtype=tf.float32))
tfAccuracyObs = tf.reduce_mean(tf.reduce_min(tf.cast(tf.equal(tfActual, tfPredicted), dtype=tf.float32), axis=1))

tfsLoss = tf.summary.scalar('RMSE', tfLoss)
tfsAccuracy = tf.summary.scalar('Accuracy', 1-tfAccuracyObs)
tfsAll = tf.summary.merge([tfsLoss, tfsAccuracy])
tfsSaver = tf.train.Saver()

tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_A', tf.get_default_graph())
print('Graph creation complete')

Graph creation complete


In [89]:
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
#tffw = tf.summary.FileWriter('D:/Jupyter/Logs/12RNN07-{0}'.format(dt_now), tf.get_default_graph())

batch_size = 1000
num_steps  = 10
num_epochs = 100
checkpoints = 500

fmtstr = 'Epoch {0} ({1:1.3} sec): \t\tVL:{2:1.3f}\t\tAC:{3:1.3f} ({4:1.3f})'
valid_batch = {tfi_x: valid_x}
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        te0 = time.perf_counter()
        for mini_x in shuffleBatches(train_x, batchSize=batch_size):
            train_batch = {tfi_x:mini_x}
            
            #l0 = tfLoss.eval(feed_dict=train_batch)
            #t0 = time.perf_counter()
            for j in range(num_steps):
                tfTrain.run(feed_dict=train_batch)
            #t1 = time.perf_counter()
            l1 = tfLoss.eval(feed_dict=train_batch)
    
        te1 = time.perf_counter()
        [lv,ac,ac0,summary] = tfs.run([tfLoss, tfAccuracyObs, tfAccuracyChar, tfsAll], feed_dict=valid_batch)
            #tffw.add_summary(summary, i)
            #if i%checkpoints == 0 and i > 0:
            #    p = tfsSaver.save(tfs, 'D:/Jupyter/mltest/Models-12RNN07/model-{0:04d}.ckpt'.format(i))
            #    print('Model saved at checkpoint: {0}'.format(p))
                             
        print(fmtstr.format(i,te1-te0,lv,ac,ac0))
    valid_r = tfs.run(tfOutR, feed_dict=valid_batch)

Epoch 0 (51.7 sec): 		VL:0.738		AC:0.003 (0.645)
Epoch 1 (51.4 sec): 		VL:0.664		AC:0.005 (0.685)
Epoch 2 (51.2 sec): 		VL:0.564		AC:0.014 (0.740)
Epoch 3 (51.2 sec): 		VL:0.514		AC:0.027 (0.767)
Epoch 4 (9.26e+02 sec): 		VL:0.480		AC:0.036 (0.785)


KeyboardInterrupt: 

In [ ]:
#5-gru broke 4-length, problems with 5-length
#6-gru broke 5-length (in 100 epochs)
